# 📈 Tuning de Estrategia RSI Divergencia v3

### 🎯 Objetivo
Evaluar, ajustar y optimizar la estrategia **RSI Divergencia v3**, que busca detectar **reversiones técnicas** en zonas de sobrecompra o sobreventa mediante señales del RSI y confirmaciones contextuales de precio, cuerpo de vela y volatilidad.

---

### ⚙️ Descripción de la Estrategia

La estrategia se basa en:

1. **RSI clásico** (default: 14 periodos)
   - Señal de **compra** cuando el RSI está por debajo del umbral de sobreventa (por defecto `rsi < 30`)
   - Señal de **venta** cuando el RSI está por encima del umbral de sobrecompra (por defecto `rsi > 70`)

2. **Confirmaciones adicionales (opcionales)**:
   - `confirmar_direccion_rsi`: exige que el RSI esté subiendo para señales de compra (o bajando para venta)
   - `confirmar_rebote_cuerpo`: requiere que el cuerpo de la vela actual sea mayor al promedio, lo cual apoya la idea de reversión con convicción
   - `usar_filtro_volatilidad`: solo emite señales si la volatilidad (ATR / precio) supera cierto umbral mínimo (`vol_threshold`)

---

### 🔬 Parámetros a evaluar

- `confirmar_direccion_rsi`: `True / False`
- `confirmar_rebote_cuerpo`: `True / False`
- `usar_filtro_volatilidad`: `True / False`
- `vol_threshold`: `[0.006, 0.008, 0.010, 0.012]`

Todos estos parámetros serán evaluados en combinación (`grid search`) para determinar cuál ofrece la mejor relación entre tasa de aciertos (`winrate`), ganancia promedio (`avg_profit`) y score compuesto.

---

### 📦 Datos y Backtest

- Se utilizarán históricos reales cargados desde archivos `.parquet`
- Se simulará cada operación en una ventana de 7 días con `TP` y `SL` definidos
- Cada combinación de parámetros será evaluada en paralelo

---

### 📊 Objetivo del Experimento

Encontrar la configuración que:
- Maximice el **score** (winrate × profit promedio)
- Mantenga una frecuencia de operación razonable
- Sea robusta ante distintas condiciones de mercado

In [2]:
import sys
sys.path.append("D:/trading")

In [7]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
from joblib import Parallel, delayed

# Cargar históricos
ruta_historicos = "D:/trading/data/historic"
historicos = {}
for archivo in os.listdir(ruta_historicos):
    if archivo.endswith(".parquet"):
        ticker = archivo.replace(".parquet", "")
        df = pd.read_parquet(os.path.join(ruta_historicos, archivo))
        historicos[ticker] = df
print(f"✅ Símbolos cargados: {len(historicos)}")

✅ Símbolos cargados: 48


In [8]:
# ✅ CELDA 2: Definición del grid de parámetros para RSI Divergencia v3
import itertools

# Combinaciones booleanas
bool_combos = list(itertools.product(
    [True, False],  # confirmar_direccion_rsi
    [True, False],  # confirmar_rebote_cuerpo
    [True, False],  # usar_filtro_volatilidad
))

# Umbrales de volatilidad
vol_thresholds = [0.006, 0.008, 0.010, 0.012]

# Combinar todas las combinaciones posibles
param_grid = list(itertools.product(bool_combos, vol_thresholds))
print(f"Total combinaciones a evaluar: {len(param_grid)}")




Total combinaciones a evaluar: 32


In [9]:
# ✅ CELDA 3: Función de simulación de operaciones por combinación
from my_modules.estrategias.v3 import rsi_divergencia_v3

def simular_combinacion(params):
    (conf_rsi, conf_cuerpo, usar_vol), vol_th = params
    resultados = []

    for ticker, df in historicos.items():
        df_signals = rsi_divergencia_v3.generar_senales(
            df,
            confirmar_direccion_rsi=conf_rsi,
            confirmar_rebote_cuerpo=conf_cuerpo,
            usar_filtro_volatilidad=usar_vol,
            vol_threshold=vol_th,
            debug=False
        )
        df_signals = df_signals[df_signals["signal"] != "hold"]
        if df_signals.empty:
            continue

        df_precio = df.copy()
        df_precio["fecha"] = pd.to_datetime(df_precio["fecha"])
        df_signals["fecha"] = pd.to_datetime(df_signals["fecha"])
        df_merged = df_precio.merge(df_signals, on="fecha")

        for _, row in df_merged.iterrows():
            fecha_entrada = row["fecha"]
            precio_entrada = row["close"]
            df_rango = df_precio[(df_precio["fecha"] > fecha_entrada) & (df_precio["fecha"] <= fecha_entrada + pd.Timedelta(days=7))]
            if df_rango.empty:
                continue

            tipo_salida = "TIMEOUT"
            fila_salida = df_rango.iloc[-1]

            for _, f in df_rango.iterrows():
                if row["signal"] == "buy":
                    if f["high"] >= precio_entrada * 1.05:
                        tipo_salida = "TP"; fila_salida = f; break
                    if f["low"] <= precio_entrada * 0.97:
                        tipo_salida = "SL"; fila_salida = f; break
                elif row["signal"] == "sell":
                    if f["low"] <= precio_entrada * 0.95:
                        tipo_salida = "TP"; fila_salida = f; break
                    if f["high"] >= precio_entrada * 1.03:
                        tipo_salida = "SL"; fila_salida = f; break

            precio_salida = fila_salida["close"]
            dias = (fila_salida["fecha"] - fecha_entrada).days
            resultado = precio_salida - precio_entrada if row["signal"] == "buy" else precio_entrada - precio_salida
            ret_pct = (precio_salida / precio_entrada - 1) * (1 if row["signal"] == "buy" else -1)
            log_ret = np.log(precio_salida / precio_entrada) * (1 if row["signal"] == "buy" else -1)

            resultados.append({
                "resultado": resultado,
                "resultado_pct": ret_pct,
                "log_ret": log_ret,
                "f_win": int(resultado > 0),
                "dias": dias
            })

    df = pd.DataFrame(resultados)
    if df.empty:
        return {
            "confirmar_direccion_rsi": conf_rsi,
            "confirmar_rebote_cuerpo": conf_cuerpo,
            "usar_filtro_volatilidad": usar_vol,
            "vol_threshold": vol_th,
            "n_trades": 0,
            "winrate": 0,
            "avg_profit": 0,
            "score": -999
        }

    return {
        "confirmar_direccion_rsi": conf_rsi,
        "confirmar_rebote_cuerpo": conf_cuerpo,
        "usar_filtro_volatilidad": usar_vol,
        "vol_threshold": vol_th,
        "n_trades": len(df),
        "winrate": df["f_win"].mean(),
        "avg_profit": df["resultado"].mean(),
        "score": df["resultado"].mean() * df["f_win"].mean()
    }


In [10]:
# ✅ CELDA 4: Ejecutar tuning en paralelo
from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

resultados = Parallel(n_jobs=-1)(
    delayed(simular_combinacion)(params) for params in tqdm(param_grid)
)

df_tuning = pd.DataFrame(resultados).sort_values("score", ascending=False).reset_index(drop=True)
print("Top combinaciones por score:")
display(df_tuning.head(10))


 47%|██████████████████████████████████████▍                                           | 15/32 [01:51<02:06,  7.44s/it]

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:21<00:00,  1.50it/s]


Top combinaciones por score:


,confirmar_direccion_rsi,confirmar_rebote_cuerpo,usar_filtro_volatilidad,vol_threshold,n_trades,winrate,avg_profit,score
0,False,False,True,0.010,5544,0.462121,-0.027588,-0.012749
1,False,False,False,0.008,5642,0.462425,-0.028923,-0.013375
2,False,False,False,0.006,5642,0.462425,-0.028923,-0.013375
3,False,False,False,0.010,5642,0.462425,-0.028923,-0.013375
4,False,False,False,0.012,5642,0.462425,-0.028923,-0.013375
5,False,False,True,0.006,5628,0.461976,-0.029113,-0.013449
6,False,False,True,0.008,5602,0.461799,-0.029232,-0.013499
7,False,False,True,0.012,5338,0.458411,-0.030482,-0.013973
8,True,True,True,0.012,741,0.404858,-0.035331,-0.014304
9,True,True,True,0.006,767,0.401565,-0.039453,-0.015843
